In [1]:
!pip install transformers datasets evaluate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 21.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
import evaluate

data_path = "drive/MyDrive/diploma/jobs-main-data.csv"


class ModelColumns:
    budget = "budget"
    hourlyRangeMin = "hourlyRangeMin"
    isHourlyPayment = "isHourlyPayment"
    country = "country"
    category = "category"
    workload = "workload"
    duration = "duration"
    clientTotalCharge = "clientTotalCharge"
    clientTotalJobsPosted = "clientTotalJobsPosted"
    clientFeedbackScore = "clientFeedbackScore"
    clientPastHires = "clientPastHires"
    isPaymentMethodVerified = "isPaymentMethodVerified"
    skills = "skills"
    target = "target"


id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
accuracy = evaluate.load("accuracy")

def prepare_dataset():
    df = pd.read_csv(data_path)

    df = df[df["status"].isin(["prelead", "in-progress"]) == False]

    def transform_status_into_target(row):
        if row["status"] == "trashed":
            return 0
        else:
            return 1

    df[ModelColumns.target] = df.apply(transform_status_into_target, axis=1)
    df = df.drop(
        columns=[
            "id",
            "uid",
            "score",
            "createdAt",
            "status",
            "postedAt",
            "query",
        ]
    )

    df.dropna(subset=[ModelColumns.country, ModelColumns.duration], inplace=True)

    # reassign jobs which do not have verified method payment, but were taken
    df.loc[
        (
            (df[ModelColumns.isPaymentMethodVerified] == 0)
            & (df[ModelColumns.target] == 1)
        ),
        ModelColumns.target,
    ] = 0

    # reassign project jobs which have budget less than 5000, but were taken
    df.loc[
        (
            (df[ModelColumns.target] == 1)
            & (df[ModelColumns.budget] < 5000)
            & (df[ModelColumns.isHourlyPayment] == 0)
        ),
        ModelColumns.target,
    ] = 0

    X = df.drop(ModelColumns.target, axis=1).copy()
    X = X[["title", "description"]]

    y = df[ModelColumns.target].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    rus = RandomUnderSampler(random_state=42, replacement=True)
    X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train, y_train)

    X_train, y_train = shuffle(
        X_train_undersampled, y_train_undersampled, random_state=42
    )

    train = X_train.copy()
    test = X_test.copy()

    train["text"] = train["title"] + "\n" + train["description"]
    test["text"] = test["title"] + "\n" + test["description"]

    train = train.drop(columns=["title", "description"])
    test = test.drop(columns=["title", "description"])

    train["label"] = y_train
    test["label"] = y_test

    train = Dataset.from_pandas(train)
    test = Dataset.from_pandas(test)

    train = train.map(preprocess_function, batched=True)
    test = test.map(preprocess_function, batched=True)

    train = train.select(range(64))
    test = test.select(range(32))

    return train, test

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


train, test = prepare_dataset()

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Map:   0%|          | 0/5014 [00:00<?, ? examples/s]

Map:   0%|          | 0/3388 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.722351,0.313333
2,No log,0.708282,0.450000


TrainOutput(global_step=64, training_loss=0.6806859970092773, metrics={'train_runtime': 66.7189, 'train_samples_per_second': 14.988, 'train_steps_per_second': 0.959, 'total_flos': 124079521577040.0, 'train_loss': 0.6806859970092773, 'epoch': 2.0})

In [18]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1713821784.cb4c43769cd2.267.1:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

events.out.tfevents.1713821521.cb4c43769cd2.267.0:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

events.out.tfevents.1713821937.cb4c43769cd2.267.2:   0%|          | 0.00/5.88k [00:00<?, ?B/s]

events.out.tfevents.1713822886.cb4c43769cd2.267.3:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

events.out.tfevents.1713823060.cb4c43769cd2.267.4:   0%|          | 0.00/5.66k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DenysZakharkevych/my_awesome_model/commit/53b44c7df377b05184b6d8c6b6a1befcef57693f', commit_message='End of training', commit_description='', oid='53b44c7df377b05184b6d8c6b6a1befcef57693f', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="my_awesome_model")

[{'label': 'POSITIVE', 'score': 0.5760232210159302}]

In [20]:
text = train[1]['text']
classifier(text)

[{'label': 'POSITIVE', 'score': 0.5359859466552734}]